In [54]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Importing data

In [63]:
lapData  = pd.read_table('./racedata.csv',sep=',')
# Preview
lapData['constructor'] = lapData['constructor'].astype(np.str0)
lapData['avglap']  = lapData['avglap'].astype(np.float64)
lapData['year']    = lapData['year'].astype(np.int64)
lapData['circuit'] = lapData['circuit'].astype(np.str0)
lapData['podium']  = lapData['podium'].astype(np.int8)
lapData.head()

,constructor,avglap,year,circuit,podium
0,Mercedes,97872.11,2014,Albert Park Grand Prix Circuit,1
1,McLaren,98341.88,2014,Albert Park Grand Prix Circuit,1
2,McLaren,98398.89,2014,Albert Park Grand Prix Circuit,1
3,Ferrari,98491.12,2014,Albert Park Grand Prix Circuit,0
4,Williams,98707.88,2014,Albert Park Grand Prix Circuit,0


### Split test / train data

In [64]:
YData = lapData['podium']
XData =  pd.get_dummies(lapData,prefix=['constructor','circuit'],columns=['constructor','circuit']).astype('float')
# XData = lapData
print(XData)
# print(pd.get_dummies(lapData.pop('avglap')))

        avglap    year  podium  constructor_Alfa Romeo  \
0     97872.11  2014.0     1.0                     0.0   
1     98341.88  2014.0     1.0                     0.0   
2     98398.89  2014.0     1.0                     0.0   
3     98491.12  2014.0     0.0                     0.0   
4     98707.88  2014.0     0.0                     0.0   
...        ...     ...     ...                     ...   
1808  91778.52  2022.0     0.0                     0.0   
1809  92118.02  2022.0     0.0                     0.0   
1810  92227.28  2022.0     0.0                     0.0   
1811  92238.14  2022.0     0.0                     0.0   
1812  92332.50  2022.0     0.0                     0.0   

      constructor_AlphaTauri  constructor_Alpine F1 Team  \
0                        0.0                         0.0   
1                        0.0                         0.0   
2                        0.0                         0.0   
3                        0.0                         0.0   
4  

In [65]:
XTrain, XTest, YTrain, YTest = train_test_split(XData, YData, test_size=0.3, random_state=2)

# Model Creation + Training

In [66]:
model = LogisticRegression()

In [67]:
model.fit(XTrain,YTrain)

LogisticRegression()

In [68]:
print(model.score(XTest,YTest))

1.0


In [69]:
predictions = model.predict(XTest)

In [70]:
predictions

array([1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,

In [71]:
print(metrics.confusion_matrix(YTest,predictions))

[[375   0]
 [  0 169]]


# Testing